In [1]:
import pandas as pd
from google.cloud import bigquery
import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import re
import string
from nltk.corpus import stopwords
import tensorflow as tf
from tensorflow import keras
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
bqclient = bigquery.Client()

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/emma_tebbe/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/emma_tebbe/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/emma_tebbe/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:

query = (
    """
    SELECT * FROM `w266-313317.final_project.tag_asset_count`
    WHERE language = 'en'
    """
)
result = bqclient.query(query).to_dataframe()
result

,tag,language,auto_generated,organization_id,asset_count
0,Dust Extractors,en,False,13770,71
1,TSC 55,en,False,13770,44
2,Social_Media_Posts,en,False,16816,33
3,Triple Teazer,en,False,13887,40
4,new,en,False,14215,62
...,...,...,...,...,...
128651,Gesture,en,True,18161,17
128652,Tan,en,True,18161,17
128653,LineLED 36X2 Imagery,en,False,18161,17
128654,Kilo Imagery,en,False,18161,17


In [3]:
result['rare_tag'] = result['asset_count'] <= 3
result['rare_tag'] = result['rare_tag'].astype(int)

In [4]:
lemmatizer = WordNetLemmatizer()
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
stop_words = set(stopwords.words('english'))


In [5]:
result_np = result[['tag','language','auto_generated','organization_id','asset_count']].to_numpy()

In [35]:
def word_preprocessing(word):
    lower = word.lower()
    punct_replacer = str.maketrans(string.punctuation, ' '*len(string.punctuation))
    rem_punct = lower.translate(punct_replacer)
    lemma = [lemmatizer.lemmatize(w) for w in nltk.word_tokenize(rem_punct)]
    rem_stop = [w for w in lemma if not w in stop_words]
    rem_digits = [re.sub('\d', '<dig>', i) for i in rem_stop]
    return rem_digits, word
lemmatized = [word_preprocessing(i[0]) for i in result_np]
lemma_df = pd.DataFrame(lemmatized, columns = ['lemmatized','tag']).drop_duplicates()

TypeError: unhashable type: 'list'

In [7]:

query = (
    """
    SELECT t1.tag, cast(t1.asset_id as string) as asset_id, 
    t1.auto_generated, 
    cast(t1.organization_id as string) as organization_id, 
    t1.source,
    t2.asset_count
    FROM `w266-313317.final_project.raw_tags` t1
    INNER JOIN `w266-313317.final_project.tag_asset_count` t2
    ON t1.tag = t2.tag
    AND t1.organization_id = t2.organization_id
    AND t1.language = t2.language
    WHERE t1.language = 'en'
    AND t2.asset_count = 1
    """
)
one_asset_tags = bqclient.query(query).to_dataframe()

In [8]:

query = (
    """
    SELECT t1.tag, cast(t1.asset_id as string) as asset_id, 
    t1.auto_generated, 
    cast(t1.organization_id as string) as organization_id, 
    t1.source,
    t2.asset_count
    FROM `w266-313317.final_project.raw_tags` t1
    INNER JOIN `w266-313317.final_project.tag_asset_count` t2
    ON t1.tag = t2.tag
    AND t1.organization_id = t2.organization_id
    AND t1.language = t2.language
    WHERE t1.language = 'en'
    AND t2.asset_count > 1
    """
)
multi_asset_tags = bqclient.query(query).to_dataframe()

KeyboardInterrupt: 

In [36]:
lemma_df

,tag,lemmatized,lemma_string
0,Dust Extractors,"[dust, extractor]",dust extractor
1,TSC 55,"[tsc, <dig><dig>]",tsc <dig><dig>
2,Social_Media_Posts,"[social, medium, post]",social medium post
3,Triple Teazer,"[triple, teazer]",triple teazer
4,new,[new],new
...,...,...,...
128651,Gesture,[gesture],gesture
128652,Tan,[tan],tan
128653,LineLED 36X2 Imagery,"[lineled, <dig><dig>x<dig>, imagery]",lineled <dig><dig>x<dig> imagery
128654,Kilo Imagery,"[kilo, imagery]",kilo imagery


In [48]:
lemma_df['lemma_string'] =  [" ".join(map(str, l)) for l in lemma_df['lemmatized']]
lemma_bq_df = lemma_df[['tag', 'lemma_string']].drop_duplicates()
lemma_bq_df = lemma_bq_df[lemma_bq_df.lemma_string.notnull() & lemma_bq_df.lemma_string.notna()]
lemma_bq_df

,tag,lemma_string
0,Dust Extractors,dust extractor
1,TSC 55,tsc <dig><dig>
2,Social_Media_Posts,social medium post
3,Triple Teazer,triple teazer
4,new,new
...,...,...
128644,LineLED 36X2,lineled <dig><dig>x<dig>
128646,LineLED 54 Wet Tube,lineled <dig><dig> wet tube
128653,LineLED 36X2 Imagery,lineled <dig><dig>x<dig> imagery
128654,Kilo Imagery,kilo imagery


In [42]:
join_lemma = lemma_bq_df.set_index('tag')
with_lemma = result.join(join_lemma, on='tag')

In [43]:
with_lemma

,tag,language,auto_generated,organization_id,asset_count,rare_tag,lemma_string
0,Dust Extractors,en,False,13770,71,0,dust extractor
1,TSC 55,en,False,13770,44,0,tsc <dig><dig>
2,Social_Media_Posts,en,False,16816,33,0,social medium post
3,Triple Teazer,en,False,13887,40,0,triple teazer
4,new,en,False,14215,62,0,new
...,...,...,...,...,...,...,...
128651,Gesture,en,True,18161,17,0,gesture
128652,Tan,en,True,18161,17,0,tan
128653,LineLED 36X2 Imagery,en,False,18161,17,0,lineled <dig><dig>x<dig> imagery
128654,Kilo Imagery,en,False,18161,17,0,kilo imagery


In [45]:
job_config = bigquery.LoadJobConfig(
    # Specify a (partial) schema. All columns are always written to the
    # table. The schema is used to assist in data type definitions.
    schema=[
        # Specify the type of columns whose type cannot be auto-detected. For
        # example the "title" column uses pandas dtype "object", so its
        # data type is ambiguous.
        bigquery.SchemaField("tag", bigquery.enums.SqlTypeNames.STRING),
        # Indexes are written if included in the schema by name.
        bigquery.SchemaField("lemma_string", bigquery.enums.SqlTypeNames.STRING),
    ],
    # Optionally, set the write disposition. BigQuery appends loaded rows
    # to an existing table by default, but with WRITE_TRUNCATE write
    # disposition it replaces the table with the loaded data.
    write_disposition="WRITE_TRUNCATE",
)
table_id = "w266-313317.final_project.tag_asset_count_lemma"
job = bqclient.load_table_from_dataframe(
    with_lemma, table_id, job_config=job_config
)  # Make an API request.
job.result()